In [1]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import deepcopy
import data_generation as dg
import time

In [533]:
graph, random_solution, total_edge_length = dg.read_scientific_instance('data\\nug_30_5.txt')
num_nodes = len(graph)

In [534]:
sols = []
values = []
iters = []
times = []

<span style="font-size: larger;">Classic local search</span>

In [535]:
def make_change_swap(graph, solution):
    new_solution = solution.copy()
    
    index1, index2 = random.sample(range(len(solution)), 2)
    
    new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]

    return new_solution

In [536]:
def make_change_inverse(graph, solution):
    new_solution = solution.copy()
    
    index1, index2 = sorted(random.sample(range(len(solution)), 2))
    
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])

    return new_solution

In [537]:
def make_change_next_permutation(graph, solution):
    #based on the classic next permutation algorithm
    #we will be using this in a modified local search when regular methods get stuck in hopes of finding
    #a better solution close to the current one
    new_solution = deepcopy(solution)
    
    n = len(new_solution)
    
    k = n - 2
    while k >= 0 and new_solution[k] >= new_solution[k + 1]:
        k -= 1

    if k == -1:
        return sorted(new_solution)

    l = n - 1
    while new_solution[k] >= new_solution[l]:
        l -= 1

    new_solution[k], new_solution[l] = new_solution[l], new_solution[k]

    new_solution[k + 1:] = reversed(new_solution[k + 1:])

    return new_solution


In [538]:
def make_change_scramble(graph, solution):
    #another change function, as the name suggest, this one will scramble a section of the solution
    #this wouldn't be the method we use if we only tested one version, but since there are plenty of change functions and combinations,
    #this one might prove most useful in larger dimension
    new_solution = deepcopy(solution)
    
    start, end = sorted(random.sample(range(len(solution)), 2))

    subset = solution[start:end+1]

    random.shuffle(subset)

    new_solution[start:end+1] = subset

    return new_solution

In [539]:
def local_search(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        #print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [540]:
start = time.time()
solution, value, iter = local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 25, 1, 6, 27, 17, 29, 10, 22, 18, 13, 8, 20, 3, 19, 15, 14, 9, 21, 16, 7, 28, 11, 24, 2, 26, 12, 23, 0, 4] 3435 455


In [541]:
start = time.time()
solution, value, iter = local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[25, 17, 29, 6, 27, 1, 3, 18, 10, 13, 22, 20, 5, 24, 28, 8, 15, 9, 21, 14, 7, 19, 0, 16, 2, 23, 11, 12, 26, 4] 3462 499


In [542]:
start = time.time()
solution, value, iter = local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[23, 12, 0, 2, 11, 24, 28, 26, 4, 7, 16, 19, 9, 14, 1, 21, 15, 29, 8, 20, 10, 18, 13, 5, 22, 27, 3, 6, 17, 25] 3443 492


In [543]:
#generating comparison table
from IPython.display import display
import pandas as pd
dim = num_nodes

results = []

best = float('inf')
best_i = -1

average = [0,0,0]

number_of_combinations = 0

solution_generation = [make_change_swap, make_change_inverse, make_change_scramble]
for i, sg in enumerate(solution_generation):
    method = sg.__name__
    results.append({'Dim': dim, 'Method': method, 'Value': values[i], 'Time': times[i], 'Best_Iter': iters[i]})

    if values[i] == best:
        if times[i] <= times[best_i]:
            best = values[i]
            best_i = i

    if values[i] < best:
        best = values[i]
        best_i = i

    average[0] += values[i]
    average[1] += times[i] 
    average[2] += iters[i]

    number_of_combinations += 1

df = pd.DataFrame(results)
#display(df)
display(df.drop('Best_Iter', axis=1)) 

,Dim,Method,Value,Time
0,30,make_change_swap,3435,0.10
1,30,make_change_inverse,3462,0.10
2,30,make_change_scramble,3443,0.11


In [544]:
print('best:', solution_generation[best_i].__name__, values[best_i], times[best_i], iters[best_i])
df_best = pd.DataFrame({'Dim': dim, 'Method':  solution_generation[best_i].__name__, 'Value': values[best_i], 'Time': times[best_i], 'Best_Iter': iters[best_i]}, index=[0])
display(df_best)

best: make_change_swap 3435 0.1 455


,Dim,Method,Value,Time,Best_Iter
0,30,make_change_swap,3435,0.1,455


In [545]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

#check if first write to the specified row
if not row_to_update.empty:
    #update specific columns in the located row
    df.loc[row_to_update.index, 'local_search'] = values[best_i]
    df.loc[row_to_update.index, 'local_search_time'] = times[best_i]

    #save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'local_search': values[best_i], 'local_search_time': times[best_i]}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [546]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations, average[2] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': dim, 'Value': average[0], 'Time': average[1], 'Best_Iter': average[2]}, index=[0])
display(df_avg)

average: [3446.67, 0.1, 482.0]


,Dim,Value,Time,Best_Iter
0,30,3446.67,0.1,482.0


In [547]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

if not row_to_update.empty:
    df.loc[row_to_update.index, 'local_search'] = average[0]
    df.loc[row_to_update.index, 'local_search_time'] = average[1]


    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'local_search': average[0], 'local_search_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)

<span style="font-size: larger;">Local search with next permutation help</span>

In [548]:
sols = []
values = []
iters = []
times = []

def local_search_with_permutation(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        #print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        else:
            perm_counter = 0
            perm_limit = num_iters / (i+1)
            while new_value >= value and perm_counter < perm_limit:    
                new_solution = make_change_next_permutation(graph, new_solution)
                new_value = dg.calculate_total_edge_length(graph, new_solution)
                perm_counter += 1

            if new_value < value:
                value = new_value
                solution = deepcopy(new_solution)

                if new_value < best_value:
                    best_i = i
                    best_value = new_value
                    best_solution = deepcopy(new_solution)
              

    return best_solution, best_value, best_i

In [549]:
start = time.time()
solution, value, iter = local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[23, 26, 0, 12, 28, 19, 24, 11, 7, 4, 9, 21, 2, 22, 16, 29, 14, 10, 15, 8, 20, 3, 5, 13, 1, 6, 17, 18, 25, 27] 3466 496


In [550]:
start = time.time()
solution, value, iter = local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[27, 29, 1, 18, 3, 25, 17, 6, 5, 22, 13, 20, 8, 15, 10, 14, 21, 9, 7, 19, 11, 23, 16, 2, 12, 28, 24, 0, 4, 26] 3422 473


In [551]:
start = time.time()
solution, value, iter = local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[24, 6, 17, 27, 18, 25, 10, 5, 13, 20, 29, 22, 8, 26, 21, 14, 3, 16, 1, 9, 0, 19, 28, 15, 11, 7, 23, 12, 2, 4] 3528 480


In [552]:
#generating comparison table
from IPython.display import display
import pandas as pd
dim = num_nodes

results = []

best = float('inf')
best_i = -1

average = [0,0,0]

number_of_combinations = 0

solution_generation = [make_change_swap, make_change_inverse, make_change_scramble]
for i, sg in enumerate(solution_generation):
    method = sg.__name__ + '_with_permutation'
    results.append({'Dim': dim, 'Method': method, 'Value': values[i], 'Time': times[i], 'Best_Iter': iters[i]})

    if values[i] == best:
        if times[i] <= times[best_i]:
            best = values[i]
            best_i = i

    if values[i] < best:
        best = values[i]
        best_i = i    

    average[0] += values[i]
    average[1] += times[i] 
    average[2] += iters[i]

    number_of_combinations += 1

df = pd.DataFrame(results)
#display(df)
display(df.drop('Best_Iter', axis=1)) 

,Dim,Method,Value,Time
0,30,make_change_swap_with_permutation,3466,0.56
1,30,make_change_inverse_with_permutation,3422,0.55
2,30,make_change_scramble_with_permutation,3528,0.61


In [553]:
print('best:', solution_generation[best_i].__name__, values[best_i], times[best_i], iters[best_i])
df_best = pd.DataFrame({'Dim': dim, 'Method':  solution_generation[best_i].__name__ + '_with_permutation', 'Value': values[best_i], 'Time': times[best_i], 'Best_Iter': iters[best_i]}, index=[0])
display(df_best)

best: make_change_inverse 3422 0.55 473


,Dim,Method,Value,Time,Best_Iter
0,30,make_change_inverse_with_permutation,3422,0.55,473


In [554]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

if not row_to_update.empty:
    df.loc[row_to_update.index, 'local_search_perm'] = values[best_i]
    df.loc[row_to_update.index, 'local_search_perm_time'] = times[best_i]

    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'local_search_perm': values[best_i], 'local_search_perm_time': times[best_i]}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [555]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations, average[2] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': dim, 'Value': average[0], 'Time': average[1], 'Best_Iter': average[2]}, index=[0])
display(df_avg)

average: [3472.0, 0.57, 483.0]


,Dim,Value,Time,Best_Iter
0,30,3472.0,0.57,483.0


In [556]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

if not row_to_update.empty:
    df.loc[row_to_update.index, 'local_search_perm'] = average[0]
    df.loc[row_to_update.index, 'local_search_perm_time'] = average[1]

    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'local_search_perm': average[0], 'local_search_perm_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)

<span style="font-size: larger;">Simulated annealing</span>

In [557]:
sols = []
values = []
iters = []
times = []

def simulated_annealing(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(1, num_iters + 1):
        #print(solution, value)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        #elif random.random() < 1 / (i**0.5):
        elif random.random() < 1 / i:
            #print('divs')
            value = new_value
            solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [558]:
start = time.time()
solution, value, iter = simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[25, 27, 1, 6, 29, 3, 18, 17, 10, 22, 13, 5, 20, 8, 15, 14, 21, 9, 16, 7, 19, 28, 11, 12, 24, 26, 0, 4, 23, 2] 3419 497


In [559]:
start = time.time()
solution, value, iter = simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[24, 11, 4, 26, 23, 0, 2, 7, 19, 12, 16, 21, 9, 28, 14, 15, 29, 18, 20, 10, 13, 8, 27, 22, 5, 3, 25, 17, 6, 1] 3442 481


In [560]:
start = time.time()
solution, value, iter = simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[6, 29, 18, 1, 27, 17, 25, 22, 3, 20, 13, 10, 8, 15, 21, 5, 28, 14, 19, 9, 7, 12, 16, 24, 0, 2, 23, 11, 4, 26] 3446 477


In [561]:
#generating comparison table
from IPython.display import display
import pandas as pd
dim = num_nodes

results = []

best = float('inf')
best_i = -1

average = [0,0,0]

number_of_combinations = 0

solution_generation = [make_change_swap, make_change_inverse, make_change_scramble]
for i, sg in enumerate(solution_generation):
    method = sg.__name__
    results.append({'Dim': dim, 'Method': 'simulated_annealing' + ', '  + method, 'Value': values[i], 'Time': times[i], 'Best_Iter': iters[i]})

    if values[i] == best:
        if times[i] <= times[best_i]:
            best = values[i]
            best_i = i

    if values[i] < best:
        best = values[i]
        best_i = i

    average[0] += values[i]
    average[1] += times[i] 
    average[2] += iters[i]

    number_of_combinations += 1

df = pd.DataFrame(results)
#display(df)
display(df.drop('Best_Iter', axis=1)) 

,Dim,Method,Value,Time
0,30,"simulated_annealing, make_change_swap",3419,0.10
1,30,"simulated_annealing, make_change_inverse",3442,0.10
2,30,"simulated_annealing, make_change_scramble",3446,0.13


In [562]:
print('best:', solution_generation[best_i].__name__, values[best_i], times[best_i], iters[best_i])
df_best = pd.DataFrame({'Dim': dim, 'Method':  'simulated_annealing' + ', '  + solution_generation[best_i].__name__, 'Value': values[best_i], 'Time': times[best_i], 'Best_Iter': iters[best_i]}, index=[0])
display(df_best)

best: make_change_swap 3419 0.1 497


,Dim,Method,Value,Time,Best_Iter
0,30,"simulated_annealing, make_change_swap",3419,0.1,497


In [563]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'sim_annealing'] = values[best_i]
    df.loc[row_to_update.index, 'sim_annealing_time'] = times[best_i]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'sim_annealing': values[best_i], 'sim_annealing_time': times[best_i]}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [564]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations, average[2] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': dim, 'Value': average[0], 'Time': average[1], 'Best_Iter': average[2]}, index=[0])
display(df_avg)

average: [3435.67, 0.11, 485.0]


,Dim,Value,Time,Best_Iter
0,30,3435.67,0.11,485.0


In [565]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'sim_annealing'] = average[0]
    df.loc[row_to_update.index, 'sim_annealing_time'] = average[1]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'sim_annealing_perm': average[0], 'sim_annealing_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)

<span style="font-size: larger;">Variable neigborhood search</span>

In [566]:
def shaking_swap(graph, solution, k):
    new_solution = deepcopy(solution)
    selected = random.sample(range(len(solution)), 2*k)
    for i in range(0,len(selected), 2):
        index1, index2 = selected[i], selected[i+1]
        new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]
    return new_solution    
    

In [567]:
def shaking_inverse(graph, solution, k):
    new_solution = deepcopy(solution)
    index1 = random.randint(0, len(solution))
    if(index1 + k > len(solution)):
        index1 = len(solution) - k
    index2 = index1 + k                       
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])  

    return new_solution

In [568]:
def vns(graph, random_solution, value, num_iters, change_func, shaking_func, local_search_func, k_min, k_max, move_prob):
    solution = deepcopy(random_solution)
    best_i = None
    for i in range(num_iters):
        for k in range(k_min, k_max):
            #print('vns: ', solution, value, i)
            #diversification
            new_solution = shaking_func(graph, solution, k)   

            new_value = dg.calculate_total_edge_length(graph, new_solution)

            #print('post shaking: ', new_solution, new_value, i)
            new_solution, new_value, _= local_search_func(graph, new_solution, total_edge_length, 10, change_func)

            if new_value < value or (new_value == value and random.random() < move_prob):
                if(new_value < value):
                    best_i = i

                value = new_value

                solution = deepcopy(new_solution)
                
    return solution, value, best_i

In [569]:
#too many combos for VNS, we will iterrate
#this soulution SHOULD NOT be used regularly, as the cell execution time will be MUCH longer than the sum of it's parts for some reason (notebook problem?)
#for example, for a certain n, if we split this into seperate cells the sum of exec time would be 3mins, this way it's around 5-6mins
#hopefully this doesn't seem to affect individual duration calculations...
n = len(graph)

oc = False

shakings = [shaking_swap, shaking_inverse]
changes_funcs = [make_change_swap, make_change_inverse, make_change_scramble]
search_funcs = [local_search, local_search_with_permutation]

number_of_combinations = 0

sols = []
values =[]
times = []

methods = []

results = []

best = float('inf')
best_i = -1

average = [0,0]

k_min = int(0.1 * num_nodes)
k_max = int(0.3 * num_nodes)
for shake in shakings:
    for change in changes_funcs:
        for search in search_funcs:
            if shake.__name__ == 'shaking_inverse':
                k_min = int(0.3 * num_nodes)
                k_max = int(0.6 * num_nodes)

            start = time.time()    
            sol, value, iter = vns(graph, 
                                   random_solution, 
                                   total_edge_length, 
                                   500, 
                                   change_func=change, 
                                   shaking_func=shake, 
                                   local_search_func = search, 
                                   k_min=k_min,
                                   k_max=k_max, 
                                   move_prob=0.4)
            end = time.time()
            duration = float("{:.2f}".format(end - start))
            
            print(shake.__name__, change.__name__, search.__name__, ' : ', sol, value, iter, '\n')
            
            k_min = int(0.1 * num_nodes)
            k_max = int(0.3 * num_nodes)

            sols.append(sol)
            values.append(value)
            iters.append(iter)
            times.append(duration)

            methods.append(str(shake.__name__ + ', ' + change.__name__ + ', ' + search.__name__))

            results.append({'Dim': num_nodes, 'Method': methods[number_of_combinations], 'Value': abs(value), 'Time': duration})

            if value == best:
                if duration <= times[best_i]:    
                    best = value
                    best_i = number_of_combinations

            if value < best:
                best = value
                best_i = number_of_combinations            

            average[0] += value
            average[1] += duration

            number_of_combinations += 1    


shaking_swap make_change_swap local_search  :  [4, 24, 28, 23, 0, 11, 26, 2, 7, 12, 16, 21, 19, 14, 9, 15, 8, 22, 27, 6, 20, 13, 25, 10, 5, 18, 1, 3, 17, 29] 3429 412 

shaking_swap make_change_swap local_search_with_permutation  :  [3, 5, 25, 18, 13, 1, 27, 10, 6, 29, 15, 17, 20, 8, 22, 14, 21, 9, 16, 24, 19, 2, 26, 12, 0, 7, 23, 11, 4, 28] 3439 352 

shaking_swap make_change_inverse local_search  :  [4, 24, 2, 12, 28, 11, 7, 26, 0, 23, 16, 9, 21, 19, 14, 8, 13, 5, 20, 29, 15, 22, 27, 10, 17, 25, 6, 1, 3, 18] 3439 471 

shaking_swap make_change_inverse local_search_with_permutation  :  [12, 0, 26, 23, 16, 4, 7, 19, 28, 9, 2, 24, 21, 11, 20, 14, 10, 8, 15, 1, 13, 29, 22, 25, 27, 6, 5, 3, 17, 18] 3450 467 

shaking_swap make_change_scramble local_search  :  [27, 1, 5, 3, 18, 6, 29, 25, 17, 13, 22, 15, 10, 20, 8, 14, 21, 9, 16, 7, 26, 11, 12, 24, 19, 2, 4, 23, 0, 28] 3424 347 

shaking_swap make_change_scramble local_search_with_permutation  :  [29, 18, 3, 17, 6, 27, 1, 10, 22, 25, 5, 13

In [570]:
df = pd.DataFrame(results)
display(df)
df.to_csv('comparison_tables/vns.csv', mode='a', header=not pd.io.common.file_exists('comparison_tables/vns.csv'), index=False)

,Dim,Method,Value,Time
0,30,"shaking_swap, make_change_swap, local_search",3429,6.53
1,30,"shaking_swap, make_change_swap, local_search_w...",3439,17.68
2,30,"shaking_swap, make_change_inverse, local_search",3439,6.57
3,30,"shaking_swap, make_change_inverse, local_searc...",3450,18.67
4,30,"shaking_swap, make_change_scramble, local_search",3424,7.12
5,30,"shaking_swap, make_change_scramble, local_sear...",3426,18.58
6,30,"shaking_inverse, make_change_swap, local_search",3425,9.78
7,30,"shaking_inverse, make_change_swap, local_searc...",3420,26.37
8,30,"shaking_inverse, make_change_inverse, local_se...",3416,9.80
9,30,"shaking_inverse, make_change_inverse, local_se...",3420,26.31


In [571]:
print('best:', methods[best_i], values[best_i], times[best_i])
df_best = pd.DataFrame({'Dim': num_nodes, 'Method':  methods[best_i], 'Value': values[best_i], 'Time': times[best_i]}, index=[0])
display(df_best)

best: shaking_inverse, make_change_inverse, local_search 3416 9.8


,Dim,Method,Value,Time
0,30,"shaking_inverse, make_change_inverse, local_se...",3416,9.8


In [572]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

if not row_to_update.empty:
    
    df.loc[row_to_update.index, 'vns'] = values[best_i]
    df.loc[row_to_update.index, 'vns_time'] = times[best_i]

    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'vns': value, 'vns_time': duration}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [573]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': num_nodes, 'Value': average[0], 'Time': average[1]}, index=[0])
display(df_avg)

average: [3427.08, 15.45]


,Dim,Value,Time
0,30,3427.08,15.45


In [574]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

if not row_to_update.empty:

    df.loc[row_to_update.index, 'vns'] = average[0]
    df.loc[row_to_update.index, 'vns_time'] = average[1]

    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'vns_perm': average[0], 'vns_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)